In [0]:
from pyspark.sql import functions as F

# Read Bronze Delta table
bronze_df = spark.table("workspace.ecommerce.events_delta")

In [0]:
events_clean = (
    bronze_df
    .filter(F.col("user_id").isNotNull())
    .filter(F.col("event_time").isNotNull())
    .dropDuplicates()
)

In [0]:
silver_df = (
    events_clean
    .groupBy("user_id")
    .agg(
        F.count("*").alias("total_events"),

        F.count(F.when(F.col("event_type") == "purchase", 1))
            .alias("purchase_count"),

        F.count(F.when(F.col("event_type") == "cart", 1))
            .alias("cart_count"),

        F.count(F.when(F.col("event_type") == "view", 1))
            .alias("view_count"),

        F.sum(
            F.when(F.col("event_type") == "purchase", F.col("price"))
        ).alias("total_spent"),

        F.avg(
            F.when(F.col("event_type") == "purchase", F.col("price"))
        ).alias("avg_purchase_price"),

        F.min("event_time").alias("first_event_time"),
        F.max("event_time").alias("last_event_time")
    )
)

In [0]:
silver_df = silver_df.dropDuplicates(["user_id"])


In [0]:
spark.sql("DROP TABLE IF EXISTS workspace.ecommerce.user_features_silver")

silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.ecommerce.user_features_silver")

In [0]:
#Validate Feature Quality

spark.sql("""
SELECT COUNT(*) 
FROM workspace.ecommerce.user_features_silver
""").show()

In [0]:
#duplicate checks
spark.sql("""
SELECT user_id, COUNT(*)
FROM workspace.ecommerce.user_features_silver
GROUP BY user_id
HAVING COUNT(*) > 1
""").show()

In [0]:
#null Check
spark.sql("""
SELECT 
    SUM(CASE WHEN total_spent IS NULL THEN 1 ELSE 0 END) AS null_total_spent,
    SUM(CASE WHEN total_events IS NULL THEN 1 ELSE 0 END) AS null_total_events
FROM workspace.ecommerce.user_features_silver
""").show()

In [0]:
#Sanity Checks
spark.sql("""
SELECT 
    MIN(total_spent),
    MAX(total_spent),
    AVG(total_spent)
FROM workspace.ecommerce.user_features_silver
""").show()

In [0]:
silver_df = spark.table("workspace.ecommerce.user_features_silver")

In [0]:
gold_df = (
    silver_df
    .withColumn(
        "conversion_rate",
        F.when(F.col("total_events") > 0,
               F.col("purchase_count") / F.col("total_events"))
    )
    .withColumn(
        "user_value_segment",
        F.when(F.col("total_spent") >= 1000, "High Value")
         .when(F.col("total_spent") >= 300, "Medium Value")
         .otherwise("Low Value")
    )
    .withColumn(
        "activity_status",
        F.when(F.col("purchase_count") > 5, "Highly Active")
         .when(F.col("purchase_count") > 0, "Active")
         .otherwise("Browsing Only")
    )
    .withColumn(
        "spend_category",
        F.when(F.col("avg_purchase_price") >= 200, "Premium Buyer")
         .when(F.col("avg_purchase_price") >= 50, "Standard Buyer")
         .otherwise("Budget Buyer")
    )
)

In [0]:
spark.sql("DROP TABLE IF EXISTS workspace.ecommerce.user_features_gold")

gold_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.ecommerce.user_features_gold")

In [0]:
#Validate gold layer
spark.sql("""
SELECT user_value_segment, COUNT(*) 
FROM workspace.ecommerce.user_features_gold
GROUP BY user_value_segment
""").show()

In [0]:
spark.sql("""
SELECT user_value_segment, COUNT(*) 
FROM workspace.ecommerce.user_features_gold
GROUP BY user_value_segment
""").show()